# Load data

In [ ]:
import torch
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Datasets path
data_path = 'data' # NOTE: Change this path to the location of your BATADAL dataset files

# Load benign train data
train_benign_data = pd.read_csv(os.path.join(data_path, 'BATADAL_dataset03.csv')) # clean data

# Load attack train data
train_attack_data = pd.read_csv(os.path.join(data_path, 'BATADAL_dataset04.csv')) # data with attacks: 6-months long; attacks 1-7
train_attack_data = train_attack_data.rename(
                                columns={' L_T1': 'L_T1', ' L_T2': 'L_T2', ' L_T3': 'L_T3', ' L_T4': 'L_T4', ' L_T5': 'L_T5', ' L_T6': 'L_T6',	' L_T7': 'L_T7',
                                            ' P_J14': 'P_J14', ' P_J422': 'P_J422', ' P_J280': 'P_J280', ' P_J269': 'P_J269', ' P_J300': 'P_J300', ' P_J256': 'P_J256', ' P_J289': 'P_J289', ' P_J415': 'P_J415', ' P_J302': 'P_J302', ' P_J306': 'P_J306', ' P_J307': 'P_J307', ' P_J317': 'P_J317',
                                            ' F_PU1': 'F_PU1', ' F_PU2': 'F_PU2', ' F_PU3': 'F_PU3', ' F_PU4': 'F_PU4', ' F_PU5': 'F_PU5', ' F_PU6': 'F_PU6', ' F_PU7': 'F_PU7', ' F_PU8': 'F_PU8', ' F_PU9': 'F_PU9', ' F_PU10': 'F_PU10', ' F_PU11': 'F_PU11', ' F_V2': 'F_V2',
                                            ' S_PU1': 'S_PU1', ' S_PU2': 'S_PU2', ' S_PU3': 'S_PU3', ' S_PU4': 'S_PU4', ' S_PU5': 'S_PU5', ' S_PU6': 'S_PU6', ' S_PU7': 'S_PU7', ' S_PU8': 'S_PU8', ' S_PU9': 'S_PU9', ' S_PU10': 'S_PU10', ' S_PU11': 'S_PU11', ' S_V2': 'S_V2'})

# Load test data - benign + attack data
test_data = pd.read_csv(os.path.join(data_path, 'BATADAL_test_dataset.csv')) # data with attacks: 3-months long; attacks 8-14

In [3]:
# Separate the DATETIME column and the features
train_benign_datetime = train_benign_data['DATETIME']
train_attack_datetime = train_attack_data['DATETIME']
test_datetime = test_data['DATETIME']

# Preprocessing: Scale data using MinMaxScaler
scaler = MinMaxScaler()
train_benign_scaled = scaler.fit_transform(train_benign_data.drop(columns=['DATETIME', 'ATT_FLAG']))
train_attack_scaled = scaler.transform(train_attack_data.drop(columns=['DATETIME', ' ATT_FLAG']))
test_scaled = scaler.transform(test_data.drop(columns=['DATETIME']))

# Convert data to PyTorch tensors
train_benign_tensor = torch.tensor(train_benign_scaled, dtype=torch.float32)
train_attack_tensor = torch.tensor(train_attack_scaled, dtype=torch.float32)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)


In [4]:
# Attack intervals
train_benign_attacks_time_intervals = []
train_attack_attacks_time_intervals = [
        {"id": 1, "start": "13/09/2016 23", "end": "16/09/2016 00"},    # Attacker changesL_T7 thresholds -->> low level in T7
        {"id": 2, "start": "26/09/2016 11", "end": "27/09/2016 10"},    # Attacker changesL_T7 thresholds -->> T2 low level in T7
        {"id": 3, "start": "09/10/2016 09", "end": "11/10/2016 20"},   # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 4, "start": "29/10/2016 19", "end": "02/11/2016 16"},   # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 5, "start": "26/11/2016 17", "end": "29/11/2016 04"},   # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 6, "start": "06/12/2016 07", "end": "10/12/2016 04"},   # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 7, "start": "14/12/2016 15", "end": "19/12/2016 04"},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
]
test_attacks_time_intervals = [
        {"id": 8, "start": "16/01/2017 00", "end": "19/01/2017 06"},    # Attacker changes L_T3 thresholds -->> low level in T3
        {"id": 9, "start": "30/01/2017 08", "end": "02/02/2017 00"},    # Attacker changes L_T2 readings -->> T2 overflow
        {"id": 10, "start": "09/02/2017 03", "end": "10/02/2017 09"},   # Malicious activation of pump PU3
        {"id": 11, "start": "12/02/2017 01", "end": "13/02/2017 07"},   # Malicious activation of pump PU3
        {"id": 12, "start": "24/02/2017 05", "end": "28/02/2017 08"},   # Attacker changes L_T2, F_V2 and S_V2, P_J14 and P_J422 -->> T2 overflow
        {"id": 13, "start": "10/03/2017 14", "end": "13/03/2017 21"},   # Attacker changes L_T7 thresholds -->> PU10/PU11 are switching on/off continuously
        {"id": 14, "start": "25/03/2017 20", "end": "27/03/2017 01"}    # Attacker changes L_T4 readings -->> T6 overflow
]

# Convert attack intervals to datetime objects
train_benign_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in train_benign_attacks_time_intervals
]
train_attack_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in train_attack_attacks_time_intervals
]
test_attacks_intervals = [
    {
        "id": interval["id"],
        "start": pd.to_datetime(interval["start"], format="%d/%m/%Y %H"),
        "end": pd.to_datetime(interval["end"], format="%d/%m/%Y %H")
    }
    for interval in test_attacks_time_intervals
]

# Attack indexes
train_benign_attacks_index_intervals = []
train_attack_attacks_index_intervals = [
        {"id": 1, "start": 1727, "end": 1776},    # Attacker changesL_T7 thresholds -->> low level in T7
        {"id": 2, "start": 2027, "end": 2050},    # Attacker changesL_T7 thresholds -->> T2 low level in T7
        {"id": 3, "start": 2337, "end": 2396},    # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 4, "start": 2827, "end": 2920},    # Attack alters L_T1 readings -->> pumps PU1/PU2 on stays on
        {"id": 5, "start": 3497, "end": 3556},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 6, "start": 3727, "end": 3820},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
        {"id": 7, "start": 3927, "end": 4036},    # Attacker reduces working speed of PU7 -->> lower water levels in T4
]
test_attacks_index_intervals = [
        {"id": 8, "start": 297, "end": 366},    # Attacker changes L_T3 thresholds -->> low level in T3
        {"id": 9, "start": 632, "end": 696},    # Attacker changes L_T2 readings -->> T2 overflow
        {"id": 10, "start": 867, "end": 897},   # Malicious activation of pump PU3
        {"id": 11, "start": 937, "end": 967},   # Malicious activation of pump PU3
        {"id": 12, "start": 1229, "end": 1328},   # Attacker changes L_T2, F_V2 and S_V2, P_J14 and P_J422 -->> T2 overflow
        {"id": 13, "start": 1574, "end": 1653},   # Attacker changes L_T7 thresholds -->> PU10/PU11 are switching on/off continuously
        {"id": 14, "start": 1940, "end": 1969}    # Attacker changes L_T4 readings -->> T6 overflow
]

train_benign_attacks_indices  = np.zeros(train_benign_tensor.shape[0])
for attack in train_benign_attacks_index_intervals:
    train_benign_attacks_indices[attack['start']:attack['end']+1] = 1
train_attack_attacks_indices  = np.zeros(train_attack_tensor.shape[0])
for attack in train_attack_attacks_index_intervals:
    train_attack_attacks_indices[attack['start']:attack['end']+1] = 1
test_attacks_indices  = np.zeros(test_tensor.shape[0])
for attack in test_attacks_index_intervals:
    test_attacks_indices[attack['start']:attack['end']+1] = 1


# Classical Methods

- Train with clean data, test on clean+anomalious data

In [5]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_reconstruction_errors(attack_indices, attack_intervals, window_size, reconstruction_errors, sufix="", plot_threshold=False, threshold=None):
    # Plotting both reconstruction errors on a single plot
    plt.figure(figsize=(12, 6))

    start_date = '2017-04-01 00:00:00'
    hourly_datetime_list = pd.date_range(start=start_date, periods=len(attack_indices), freq='h')
    if window_size > 1:
        adjusted_indices = hourly_datetime_list[:-window_size+1] # Adjust indices to account for the window size - First-Point Strategy
    else:
        adjusted_indices = hourly_datetime_list
    for (errors, label, color) in reconstruction_errors:
        plt.plot(adjusted_indices, errors, label=label, color=color)

    # Shading attack intervals
    for interval in attack_intervals:
        start_idx = interval["start"]
        end_idx = interval["end"]

        # Adjust the indices to account for the window size
        if start_idx >= window_size and end_idx >= window_size:
            adjusted_start_idx = start_idx - window_size
            adjusted_end_idx = end_idx - window_size
            plt.axvspan(adjusted_indices[adjusted_start_idx], adjusted_indices[adjusted_end_idx], color='red', alpha=0.3)
        elif start_idx < window_size and end_idx >= window_size:
            adjusted_start_idx = 0
            adjusted_end_idx = end_idx - window_size
            plt.axvspan(adjusted_indices[adjusted_start_idx], adjusted_indices[adjusted_end_idx], color='red', alpha=0.3)
    
    # Plot threshold
    if plot_threshold:
      plt.axhline(y=threshold, color='red', linestyle='--', label="Threshold")

    # Add labels, legend, and title
    plt.yscale('log')
    plt.xlabel('Date')
    plt.ylabel('Reconstruction Error')
    plt.title(f"Reconstruction Error Over Time{sufix}")
    plt.show()

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

# Train Isolation Forest on the training data
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
iso_forest.fit(train_benign_scaled)

# Predict anomaly scores on the test data
iso_forest_anomaly_scores = -iso_forest.decision_function(test_scaled) # Multiply by -1 to convert to anomaly scores (higher should mean more anomalous)

# Convert anomaly scores to a pandas Series
iso_forest_anomaly_scores_series = pd.Series(iso_forest_anomaly_scores)
# Apply moving average smoothing (window size of 3, can be adjusted)
iso_forest_smoothed_scores = iso_forest_anomaly_scores_series.rolling(window=3, min_periods=1).mean()

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(iso_forest_smoothed_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(abs(iso_forest_smoothed_scores), f"Isolation Forest", "green")], 
                           sufix=f" - Isolation Forest Anomaly Score")

## One Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

# Train One-Class SVM on the training data
one_class_svm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
one_class_svm.fit(train_benign_scaled)

# Predict anomaly scores on the test data
svm_scores = -one_class_svm.decision_function(test_scaled) # Multiply by -1 to convert to anomaly scores (higher should mean more anomalous)

# Convert anomaly scores to a pandas Series
svm_scores_series = pd.Series(svm_scores)
# Apply moving average smoothing (window size of 3, can be adjusted)
smoothed_svm_scores = svm_scores_series.rolling(window=3, min_periods=1).mean()

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(smoothed_svm_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(abs(smoothed_svm_scores), f"One-Class SVM", "green")],
                           sufix=f" - One-Class SVM Anomaly Score")


##  Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Train Local Outlier Factor on the training data
lof = LocalOutlierFactor(n_neighbors=20, contamination='auto', novelty=True)
lof.fit(train_benign_scaled)

# Predict anomaly scores on the test data
lof_anomaly_scores = -lof.decision_function(test_scaled) # Multiply by -1 to convert to anomaly scores (higher should mean more anomalous)

# Convert anomaly scores to a pandas Series
lof_scores_series = pd.Series(lof_anomaly_scores)
# Apply moving average smoothing (window size of 3, can be adjusted)
smoothed_lof_scores = lof_scores_series.rolling(window=3, min_periods=1).mean()

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(smoothed_lof_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(abs(smoothed_lof_scores), f"Local Outlier Factor", "green")],
                           sufix = f" - Local Outlier Factor Anomaly Score")

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train KNN on the training data
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(train_benign_scaled, np.zeros(len(train_benign_scaled)))

# Predict anomaly scores on the test data
distances, _ = knn.kneighbors(test_scaled)
knn_scores = distances.mean(axis=1)

# Convert anomaly scores to a pandas Series
knn_scores_series = pd.Series(knn_scores)
# Apply moving average smoothing (window size of 3, can be adjusted)
smoothed_knn_scores = knn_scores_series.rolling(window=3, min_periods=1).mean()

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(smoothed_knn_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(abs(smoothed_knn_scores), f"KNN", "green")],
                           sufix = f" - KNN Anomaly Score")


## Gaussian Mixture Models (GMM)

In [ ]:
from sklearn.mixture import GaussianMixture

# Train Gaussian Mixture Model (GMM) on the training data
gmm = GaussianMixture(n_components=2, covariance_type='full', random_state=42)
gmm.fit(train_benign_scaled)

# Compute log-likelihood of the test data to determine anomaly scores
log_likelihood = gmm.score_samples(test_scaled)
gmm_scores = -log_likelihood  # Higher negative log-likelihood indicates higher anomaly

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(gmm_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(abs(gmm_scores), f"GMM", "green")],
                           sufix = f" - GMM Anomaly Score")


## GAN-Based Anomaly Detection

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the generator model
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Sigmoid()  # To generate data similar to the original range (0 to 1)
        )

    def forward(self, x):
        return self.model(x)

# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()  # To classify real vs fake
        )

    def forward(self, x):
        return self.model(x)

# Parameters
input_dim = train_benign_scaled.shape[1]
latent_dim = 32
batch_size = 64
num_epochs = 100
learning_rate = 0.0002
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert data to PyTorch tensors
train_tensor = torch.tensor(train_benign_scaled, dtype=torch.float32)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)

# Define DataLoader for training
train_loader = DataLoader(TensorDataset(train_tensor), batch_size=batch_size, shuffle=True)

# Initialize models
generator = Generator(latent_dim, input_dim).to(device)
discriminator = Discriminator(input_dim).to(device)

# Loss and optimizers
loss_function = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate)

# Training loop
real_labels = torch.ones(batch_size, 1).to(device)
fake_labels = torch.zeros(batch_size, 1).to(device)

for epoch in range(num_epochs):
    for data in train_loader:
        # Train Discriminator
        real_data = data[0].to(device)
        batch_size_current = real_data.size(0)

        # Create labels for current batch size (as the last batch may be smaller)
        real_labels_current = torch.ones(batch_size_current, 1).to(device)
        fake_labels_current = torch.zeros(batch_size_current, 1).to(device)

        # Train with real data
        outputs = discriminator(real_data)
        d_loss_real = loss_function(outputs, real_labels_current)

        # Train with fake data
        z = torch.randn(batch_size_current, latent_dim).to(device)
        fake_data = generator(z)
        outputs = discriminator(fake_data.detach())
        d_loss_fake = loss_function(outputs, fake_labels_current)

        # Total discriminator loss
        d_loss = d_loss_real + d_loss_fake

        # Backprop and optimize discriminator
        optimizer_d.zero_grad()
        d_loss.backward()
        optimizer_d.step()

        # Train Generator
        z = torch.randn(batch_size_current, latent_dim).to(device)
        fake_data = generator(z)
        outputs = discriminator(fake_data)
        g_loss = loss_function(outputs, real_labels_current)

        # Backprop and optimize generator
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}")

# Anomaly detection using reconstruction error
GAN_anomaly_scores = []

with torch.no_grad():
    for x in test_tensor:
        # Generate a fake version of the input
        x = x.to(device)
        z = torch.randn(1, latent_dim).to(device)
        generated_x = generator(z)

        # Calculate anomaly score as reconstruction error (MSE between real and generated data)
        anomaly_score = torch.mean((x - generated_x) ** 2).item()
        GAN_anomaly_scores.append(anomaly_score)

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(GAN_anomaly_scores)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(GAN_anomaly_scores, f"GAN", "green")],
                           sufix = f" - GAN Anomaly Score")


## Mahalanobis Distance in Latent Space

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.covariance import EmpiricalCovariance

# Define Autoencoder for Latent Space Representation
class LatentAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=18):
        super(LatentAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()  # To ensure output values are between 0 and 1
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Set up parameters
input_dim = train_benign_scaled.shape[1]
encoding_dim = 18
num_epochs = 50
learning_rate = 0.001

# Initialize model, optimizer, and loss function
model = LatentAutoencoder(input_dim, encoding_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Convert data to PyTorch tensors
train_tensor = torch.tensor(train_benign_scaled, dtype=torch.float32)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)

# Define DataLoader for training and testing
batch_size = 64
train_loader = DataLoader(TensorDataset(train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(test_tensor), batch_size=batch_size, shuffle=False)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training loop for Autoencoder
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data in train_loader:
        x = data[0].to(device)

        # Forward pass
        encoded_x, decoded_x = model(x)
        loss = criterion(decoded_x, x)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Calculate mean and covariance of latent space for Mahalanobis distance
model.eval()
latent_representations = []

with torch.no_grad():
    for data in train_loader:
        x = data[0].to(device)
        encoded_x, _ = model(x)
        latent_representations.append(encoded_x.cpu().numpy())

# Combine all latent representations into a single array
latent_representations = np.vstack(latent_representations)

# Fit an empirical covariance model to the latent representations
mean_vector = np.mean(latent_representations, axis=0)
cov_model = EmpiricalCovariance().fit(latent_representations)

# Function to compute Mahalanobis distance
def mahalanobis_distance(x, mean, cov):
    diff = x - mean
    distance = np.sqrt(np.dot(np.dot(diff, np.linalg.inv(cov)), diff.T))
    return distance

# Testing the Autoencoder on test data and calculating Mahalanobis distance in latent space
mahalanobis_distances = []

with torch.no_grad():
    for data in test_loader:
        x = data[0].to(device)
        encoded_x, _ = model(x)
        encoded_x = encoded_x.cpu().numpy()
        for encoded_point in encoded_x:
            dist = mahalanobis_distance(encoded_point, mean_vector, cov_model.covariance_)
            mahalanobis_distances.append(dist)

# Plot the anomaly scores over time using the test set
adjusted_indices = list(range(len(mahalanobis_distances)))
plot_reconstruction_errors(attack_indices=test_datetime, 
                           attack_intervals=test_attacks_index_intervals, 
                           window_size=1, 
                           reconstruction_errors=[(mahalanobis_distances, f"Mahalanobis Distance", "green")],
                           sufix = f" - Mahalanobis Distance in Latent Space")


# Evaluation Results

In [13]:
def find_segments(arr):
  """
  Finds segments of consecutive True values in a boolean array.

  Args:
    arr: A boolean NumPy array.

  Returns:
    A list of tuples, where each tuple represents a segment and contains the
    start and end indices (inclusive) of the segment.
  """
  segments = []
  start = -1
  for i in range(len(arr)):
    if arr[i] and start == -1:
      start = i
    elif not arr[i] and start != -1:
      segments.append((start, i - 1, i - start))
      start = -1
  if start != -1:
    segments.append((start, len(arr) - 1, len(arr)))
  return segments


In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import numpy as np

def compute_metrics_scores(reconstruction_errors, test_ground_truth_labels, method_name):

  # Calculate the best threshold to detect attacks using F1 score
  precision, recall, thresholds = precision_recall_curve(test_ground_truth_labels, reconstruction_errors)
  # Compute ROC curve
  fpr, tpr, _ = roc_curve(test_ground_truth_labels, reconstruction_errors)
  # Compute AUC (Area Under Curve)
  roc_auc = auc(fpr, tpr)

  # Calculate F1 score for each threshold
  f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
  best_threshold_idx = np.argmax(f1_scores)
  best_threshold = thresholds[best_threshold_idx]
  best_f1_score = f1_scores[best_threshold_idx]

  # Detect attacks using the best threshold
  detected_attacks = reconstruction_errors > best_threshold

  # TP
  TP = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 1))
  # TN
  TN = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 0))
  # FP
  FP = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 0))
  # FN
  FN = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 1))

  # TPR
  TPR = TP/(TP+FN)
  # TNR
  TNR = TN/(FP+TN)

  # Sclf
  Sclf = (TPR + TNR)/2
  # Sttd
  Sttd = None

  # Calculate the percentage of missed attack samples (False Negatives)
  total_attack_samples = np.sum(test_ground_truth_labels == 1)
  missed_attacks = np.sum((detected_attacks == 0) & (test_ground_truth_labels == 1))
  missed_attack_percentage = (missed_attacks / total_attack_samples) * 100

  # Calculate the percentage of wrongly detected samples (False Positives)
  total_clean_samples = np.sum(test_ground_truth_labels == 0)
  wrongly_detected = np.sum((detected_attacks == 1) & (test_ground_truth_labels == 0))
  wrongly_detected_percentage = (wrongly_detected / total_clean_samples) * 100

  # Add results to the table
  new_row = {
      "num_of_detected": 1,
      "TP": TP,
      "FP": FP,
      "TN": TN,
      "FN": FN,
      "TPR": TPR,
      "TNR": TNR,
      "S": None, #y*Sttd+(1-y)*Sclf
      "Sttd": Sttd,
      "Sclf": Sclf,
      "total_attacks": total_attack_samples,
      "total_clean": total_clean_samples,
      "missed_attacks": missed_attack_percentage,
      "wrongly_detected": wrongly_detected_percentage,
      "accuracy": (TP+TN)/(TP+TN+FP+FN),
      "f1_scores": f1_scores,
      "best_f1_score": best_f1_score,
      "best_threshold": best_threshold,
      "precisuons": precision,
      "recalls": recall,
      "fpr": fpr,
      "tpr": tpr,
      "roc_auc": roc_auc,
  }
  new_row_df = pd.DataFrame([new_row], index=[method_name])

  return new_row_df, detected_attacks, precision, recall



In [ ]:
iso_forest_results, iso_forest_detected_attacks, iso_forest_precision, iso_forest_recall = compute_metrics_scores(abs(iso_forest_smoothed_scores), test_attacks_indices, "Isolation Forest")
svm_results, svm_detected_attacks, svm_precision, svm_recall = compute_metrics_scores(abs(smoothed_svm_scores), test_attacks_indices, "One-Class SVM")
lof_results, lof_detected_attacks, lof_precision, lof_recall = compute_metrics_scores(abs(smoothed_lof_scores), test_attacks_indices, "Local Outlier Factor")
knn_results, knn_detected_attacks, knn_precision, knn_recall = compute_metrics_scores(abs(smoothed_knn_scores), test_attacks_indices, "KNN")
gmm_results, gmm_detected_attacks, gmm_precision, gmm_recall = compute_metrics_scores(abs(gmm_scores), test_attacks_indices, "GMM")
GAN_anomaly_results, GAN_anomaly_detected_attacks, GAN_anomaly_precision, GAN_anomaly_recall = compute_metrics_scores(GAN_anomaly_scores, test_attacks_indices, "GAN")
mahalanobis_distances_results, mahalanobis_distances_detected_attacks, mahalanobis_distances_precision, mahalanobis_distances_recall = compute_metrics_scores(mahalanobis_distances, test_attacks_indices, "Mahalanobis Distance in Latent Space")

pd.concat([iso_forest_results,
           svm_results,
           lof_results,
           knn_results,
           gmm_results,
           GAN_anomaly_results,
           mahalanobis_distances_results
])

In [ ]:
detected_segments_for_method = {}
real_segments_for_method = {}
removed_real_segments_for_method = {}

for detected_attacks, method in [
    (iso_forest_detected_attacks, 'iso_forest'),
    (svm_detected_attacks, 'svm'),
    (lof_detected_attacks, 'lof'),
    (knn_detected_attacks, 'knn'),
    (gmm_detected_attacks, 'gmm'),
    (GAN_anomaly_detected_attacks, 'GAN_anomaly'),
    (mahalanobis_distances_detected_attacks, 'mahalanobis_distances')
]:

  print("#"*len(f'#####   {method}   #####'))
  print(f"#####   {method}   #####")
  print("#"*len(f'#####   {method}   #####'))

  detected_segments = find_segments(detected_attacks)
  real_segments = find_segments(test_attacks_indices)
  removed_real_segments_for_method[method] = 0
  print(f"Detected segments: {detected_segments}")
  print(f"Real segments: {real_segments}")

  i = 0
  j = 0
  detected_segments_tmp = []
  real_segments_tmp = []
  while i<len(detected_segments) and j<len(real_segments):
      if detected_segments[i][1] <= real_segments[j][0]:
          i+=1
      elif (detected_segments[i][0] < real_segments[j][1]) and (real_segments[j][0] < detected_segments[i][1]):
          detected_segments_tmp.append(detected_segments[i])
          real_segments_tmp.append(real_segments[j])
          i+=1
          j+=1
          while i<len(detected_segments) and (detected_segments[i][1] < real_segments[j][0]):
              i+=1
          while j<len(real_segments) and (real_segments[j][1] < detected_segments[i][0]):
              removed_real_segments_for_method[method]+=1
              j+=1
      elif real_segments[j][1] <= detected_segments[i][0] :
          removed_real_segments_for_method[method]+=1
          j+=1
  detected_attacks_tmp  = np.zeros(detected_attacks.shape[0])
  for attack in detected_segments_tmp:
      detected_attacks_tmp[attack[0]:attack[1]+1] = 1
  real_attacks_tmp  = np.zeros(test_attacks_indices.shape[0])
  for attack in real_segments_tmp:
          real_attacks_tmp[attack[0]:attack[1]+1] = 1

  detected_segments_for_method[method] = detected_attacks_tmp
  real_segments_for_method[method] = real_attacks_tmp


In [ ]:
for detected_attacks_len, method, results in [
    (len(find_segments(iso_forest_detected_attacks)), 'iso_forest', iso_forest_results),
    (len(find_segments(svm_detected_attacks)), 'svm', svm_results),
    (len(find_segments(lof_detected_attacks)), 'lof', lof_results),
    (len(find_segments(knn_detected_attacks)), 'knn', knn_results),
    (len(find_segments(gmm_detected_attacks)), 'gmm', gmm_results),
    (len(find_segments(GAN_anomaly_detected_attacks)), 'GAN_anomaly', GAN_anomaly_results),
    (len(find_segments(mahalanobis_distances_detected_attacks)), 'mahalanobis_distances', mahalanobis_distances_results)
]:

  print("#"*len(f'#####   {method}   #####'))
  print(f"#####   {method}   #####")
  print("#"*len(f'#####   {method}   #####'))

  detected_segments = detected_segments_for_method[method]
  real_segments = real_segments_for_method[method]
  assert len(detected_segments) == len(real_segments), f"Segments lists must have the same length [{len(detected_segments)}<>{len(real_segments)}]."

  results['num_of_detected'] = detected_attacks_len
  
  ttd = 0
  for detected_segment, real_segment in zip(detected_segments, real_segments):
    ttd = ttd + max((detected_segment[0] - real_segment[0]), 0)/real_segment[2]
  ttd = ttd + removed_real_segments_for_method[method]
  Sttd = 1-ttd/len(find_segments(test_attacks_indices))

  gama = 0.5
  Sclf = results['Sclf'].values[0]
  S = gama * Sttd + (1-gama) * Sclf

  print(f"Sttd = {Sttd}, S = {S}")
  results['Sttd'] = Sttd
  results['S'] = S

## Summary

In [ ]:
pd.concat([iso_forest_results,
           svm_results,
           lof_results,
           knn_results,
           gmm_results,
           GAN_anomaly_results,
           mahalanobis_distances_results
])

In [ ]:
# Plot the values
import matplotlib.pyplot as plt

# visualize
f, ax = plt.subplots(1,figsize=(7,7))

lw = 3
font_size = 15

ax.plot(knn_results["fpr"].iloc[0], knn_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("KNN"))
ax.plot(mahalanobis_distances_results["fpr"].iloc[0], mahalanobis_distances_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("Mahalanobis Distance"))
ax.plot(gmm_results["fpr"].iloc[0], gmm_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("GMM"))
ax.plot(svm_results["fpr"].iloc[0], svm_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("One-Class SVM"))
ax.plot(GAN_anomaly_results["fpr"].iloc[0], GAN_anomaly_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("GAN"))
ax.plot(iso_forest_results["fpr"].iloc[0], iso_forest_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("Isolation Forest"))
ax.plot(lof_results["fpr"].iloc[0], lof_results["tpr"].iloc[0], linestyle = None, linewidth=lw, label='{}'.format("LOF"))

# axes label size
ax.set_xlabel("False Positive Rate", fontsize=font_size)
ax.set_ylabel("True Positive Rate", fontsize=font_size)

# Tick label size
ax.tick_params(axis='x', labelsize=font_size)
ax.tick_params(axis='y', labelsize=font_size)

ax.set_title("ROC curves")
ax.legend(fontsize=font_size)
ax.grid(True)
plt.show()
